In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from statsmodels import api as sm
from scipy import stats
from sklearn.metrics import mean_squared_error
import os
from sklearn.model_selection import train_test_split
from skimage.feature import hog
from skimage.color import rgb2gray

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import Sequential
from tensorflow.keras import activations
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from tensorboard import notebook

from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.sequence import pad_sequences

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import sklearn
import scipy.io
import numpy
import cv2

from sklearn.svm import SVC,NuSVC
from sklearn.model_selection import GridSearchCV

In [5]:
root_dir = os.path.abspath(os.getcwd())+"/256_ObjectCategories/"
#add the arrays that we can use for training and testing
main_x = []
main_y = []
prev_file = [""]
current_index = 0
for subdir, dirs, files in os.walk(root_dir):
    for dir in dirs:
        current_dir = os.path.join(root_dir) + dir
        after = str(current_dir.split(".",1)[1])
        for file in os.listdir(current_dir):
            filename = os.fsdecode(file)
            if filename.endswith(".jpg"):
                current_file = cv2.imread(os.path.join(current_dir,filename),0)
                current_file = cv2.resize(current_file, dsize=(54, 140), interpolation=cv2.INTER_CUBIC)
                main_x.append(current_file)
                main_y.append(current_index)
                continue
            else:
                continue
        current_index = current_index+1
    break
images_x = np.array(main_x)
images_y = np.array(main_y)
print(np.shape(images_x))

(30607, 140, 54)


In [ ]:
print(images_y.dtype)
fig = plt.figure(figsize=[10,10])
for i in range(100):
    ax = fig.add_subplot(10,10,i+1)
    ax.imshow(x_train[i,:,:])

In [ ]:
temp_x_train, x_test, temp_y_train, y_test = train_test_split(images_x,images_y,test_size=0.2, random_state=42)
x_train, x_val, y_train, y_val = train_test_split(temp_x_train,temp_y_train,test_size=0.2,random_state=42)
print(numpy.shape(x_train))
print(numpy.shape(x_test))
print(numpy.shape(x_val))

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report, roc_curve,auc
def eval_model(model):
    #predict Test Accuracy
    test_scores = model.evaluate(test,test_y,verbose=2)
    print("Testing Loss: "+str(test_scores[0]))
    print("Testing Accuracy: "+str(test_scores[1]))

    #display the ROC Curve
    classes = numpy.unique(train_y)
    fig = plt.figure(figsize=[10,10])
    ax = fig.add_subplot(1,1,1)
    for i in range(10):
        fpr,tpr,_=roc_curve(test_y,pred[:,i],pos_label=classes[i])
        auc_score = auc(fpr,tpr)
        ax.plot(fpr,tpr,label='%s: %0.2f' % (classes[i],auc_score))
    ax.legend()

    #confusion matrix
    fig = plt.figure(figsize=[10,10])
    ax = fig.add_subplot(1,1,1)
    pred = model.predict(test)
    indexes = tf.argmax(pred,axis=1)
    cm = confusion_matrix(test_y,indexes)
    c = ConfusionMatrixDisplay(cm,display_labels=range(10))
    c.plot(ax=ax)
    ax.set_title('Testing Performance')
    eval_model(new_model)


In [2]:
root_dir = os.path.abspath(os.getcwd())+"/256_ObjectCategories/"
#add the arrays that we can use for training and testing
main_x = []
main_y = []
prev_file = [""]
current_index = 0
for subdir, dirs, files in os.walk(root_dir):
    for dir in dirs:
        current_dir = os.path.join(root_dir) + dir
        after = str(current_dir.split(".",1)[1])
        for file in os.listdir(current_dir):
            filename = os.fsdecode(file)
            if filename.endswith(".jpg"):
                current_file = cv2.imread(os.path.join(current_dir,filename),0)
                current_file = cv2.resize(current_file, dsize=(54, 140), interpolation=cv2.INTER_CUBIC)
                main_x.append(current_file.flatten())
                main_y.append(current_index)
                continue
            else:
                continue
        current_index = current_index+1
    break
images_x = np.array(main_x)
images_y = np.array(main_y)
print(np.shape(images_x))

(30607, 7560)


In [ ]:
print(images_y.shape)

In [6]:
temp_x_train, x_test, temp_y_train, y_test = train_test_split(images_x,images_y,test_size=0.1, random_state=42)
x_train, x_val, y_train, y_val = train_test_split(temp_x_train,temp_y_train,test_size=0.1,random_state=42)
print(numpy.shape(x_train))
print(numpy.shape(x_test))
print(numpy.shape(x_val))
print(images_y[0])
print(images_y[99])

(24791, 140, 54)
(3061, 140, 54)
(2755, 140, 54)
0
1


In [ ]:
print(y_train[0])
print(y_train[98])

In [4]:
#setup the SVM
#grid search cross validation has already been imported ready for hyperparameter tuning
svm = SVC(class_weight='balanced')
#add some stuff for some shitty hyperparameter tuning here. You guys figure out the smarter way to do it. I belive the smarter way is using the range functionality to narrow down the range of values but you will need to investigate
#param_grid = [
#    {'C':[0.1,1,10,100],'kernel':['linear']},
#    {'C':[0.1,1,10,100],'gamma':[0.1,0.01,0.001],'kernel':['rbf']},
#    {'C':[0.1,1,10,100],'degree':[3,4,5],'kernel':['poly']},
#]
#grid_search = GridSearchCV(svm, param_grid)
#grid_search.fit(x_train,y_train)
#grid_search.cv_results_
#best_system = np.argmin(grid_search.cv_results_['rank_test_score'])
#params = grid_search.cv_results_['params'][best_system]
#print(params)
#svm = SVC().set_params(**params)
svm.fit(x_val,y_val)

SVC(class_weight='balanced')

In [9]:
#build a DCNN real quick
num_epochs = 50

def build_model(num_classes,output_activation=None):
    inputs = keras.Input(shape=(140,54,1),name='img')

    x = layers.Conv2D(filters=8, kernel_size=(3,3),padding='same',activation='relu')(inputs)
    x = layers.Conv2D(filters=8,kernel_size=(3,3),padding='same',activation=None)(x)

    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)

    x = layers.SpatialDropout2D(0.2)(x)

    x = layers.MaxPool2D(pool_size=(2,2))(x)

    x = layers.Conv2D(filters=8, kernel_size=(3,3),padding='same',activation='relu')(x)
    x = layers.Conv2D(filters=8,kernel_size=(3,3),padding='same',activation=None)(x)

    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)

    x = layers.SpatialDropout2D(0.2)(x)

    x = layers.MaxPool2D(pool_size=(2,2))(x)

    x = layers.Conv2D(filters=8, kernel_size=(3,3),padding='same',activation='relu')(x)
    x = layers.Conv2D(filters=8,kernel_size=(3,3),padding='same',activation=None)(x)

    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)

    x = layers.SpatialDropout2D(0.2)(x)

    x = layers.Flatten()(x)

    x = layers.Dense(256,activation='relu')(x)
    x = layers.Dropout(0.5)(x)
    x = layers.Dense(64,activation='relu')(x)
    outputs = layers.Dense(num_classes,activation=output_activation)(x)

    model_cnn = keras.Model(inputs=inputs, outputs=outputs, name='CNN')

    return model_cnn

model_cnn = build_model(257)
model_cnn.summary()

Model: "CNN"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
img (InputLayer)             [(None, 140, 54, 1)]      0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 140, 54, 8)        80        
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 140, 54, 8)        584       
_________________________________________________________________
batch_normalization_3 (Batch (None, 140, 54, 8)        32        
_________________________________________________________________
activation_3 (Activation)    (None, 140, 54, 8)        0         
_________________________________________________________________
spatial_dropout2d_3 (Spatial (None, 140, 54, 8)        0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 70, 27, 8)         0       

In [8]:
model_cnn.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(),
optimizer=keras.optimizers.Adam(),
metrics=['accuracy'])
history = model_cnn.fit(x_train,y_train,
batch_size=256,
epochs=num_epochs,
validation_data=(x_test,y_test))

Epoch 1/50


InvalidArgumentError:  Received a label value of 256 which is outside the valid range of [0, 256).  Label values: 158 199 128 154 237 255 33 189 233 243 231 89 231 209 167 0 13 158 201 238 57 172 242 47 253 153 23 81 237 65 99 183 161 102 120 138 182 256 116 144 237 56 104 105 36 214 64 174 174 58 34 40 194 151 177 194 126 144 71 126 135 195 247 71 81 160 92 136 157 65 108 39 67 52 6 143 128 12 42 170 49 154 58 31 153 138 49 35 12 10 68 213 117 168 144 211 95 9 221 108 129 147 46 184 13 79 192 104 79 147 155 198 175 176 256 211 138 39 143 95 191 37 256 94 231 243 119 234 144 163 200 163 199 95 155 183 15 199 91 95 136 232 110 4 91 136 68 54 188 127 71 143 67 66 256 121 144 52 1 144 4 217 96 231 154 254 2 36 84 159 180 168 45 208 9 254 234 39 54 244 18 95 135 39 66 161 218 91 66 142 237 224 252 236 206 240 96 209 209 144 157 157 49 132 8 147 256 235 213 243 90 220 146 115 180 10 48 172 151 214 89 213 21 24 252 12 38 169 180 256 125 191 140 140 147 90 60 1 192 111 176 196 159 144 135 119